# 🚕 NYC Taxi Streaming Demo - Databricks Free Edition

✨ **Compatível com Databricks Free Edition (SEM DBFS)**

Este notebook demonstra **Structured Streaming** usando o dataset público **NYC Taxi** disponível nativamente no Databricks.

## 🎯 Objetivos da Demonstração
- Processar dados reais de táxis de NYC com streaming
- Usar **maxFilesPerTrigger=1** para visualizar processamento gradual em batches
- Demonstrar agregações em tempo real (viagens por região, receita por tipo de pagamento)
- Salvar resultados diretamente em **Delta Tables** (sem DBFS)
- Visualizar métricas completas para ensino

## 📊 Dataset: samples.nyctaxi.trips
- **Fonte:** Unity Catalog do Databricks (já disponível!)
- **Registros:** Milhões de viagens de táxi em NYC
- **Período:** Dados históricos de táxis amarelos e verdes
- **Campos:** pickup/dropoff datetime, locations, distances, fares, payment types

## ✅ Vantagens desta abordagem
- ❌ Sem Kafka externo (Free Edition tem rede restrita)
- ❌ Sem DBFS (Public DBFS root está desabilitado)
- ✅ Usa dados REAIS (não sintéticos)
- ✅ 100% nativo do Databricks com Delta Tables
- ✅ Zero configuração de rede ou filesystem
- ✅ Perfeito para demonstrações educacionais

## 1️⃣ Setup e Configuração (SEM DBFS)

Vamos configurar o streaming **diretamente** da tabela Unity Catalog, sem usar DBFS.

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

# Configurações do streaming
TABLE_SOURCE = "samples.nyctaxi.trips"
OUTPUT_TABLE = "default.nyctaxi_streaming_results"  # Tabela Delta gerenciada
CHECKPOINT_LOCATION = "/tmp/nyctaxi_checkpoint"  # Checkpoint temporário (não precisa DBFS público)

print("🚕 NYC TAXI STREAMING DEMO (SEM DBFS)")
print("=" * 80)
print(f"📊 Fonte:      {TABLE_SOURCE} (Unity Catalog)")
print(f"💾 Destino:    {OUTPUT_TABLE} (Delta Table)")
print(f"🔖 Checkpoint: {CHECKPOINT_LOCATION}")
print("=" * 80)

print("\n✅ Abordagem:")print("=" * 80)

print("   • Ler diretamente de Unity Catalog com readStream")print("   • Visualizar métricas com foreachBatch")

print("   • Processar batches com maxFilesPerTrigger")print("   • Salvar em Delta Table gerenciada")

## 2️⃣ Explorar Dados da Tabela NYC Taxi

Vamos primeiro entender os dados disponíveis na tabela.

In [ ]:
print("📊 Explorando samples.nyctaxi.trips...")

# Ler amostra da tabela NYC Taxi
nyctaxi_sample = spark.table(TABLE_SOURCE).limit(10)

print("\n📋 Schema dos dados:")
nyctaxi_sample.printSchema()

print("\n🔍 Amostra dos primeiros registros:")
nyctaxi_sample.show(5, truncate=False)

# Contar total disponível (pode demorar)

try:    print("\n⚠️  Contagem total desabilitada para acelerar execução")

    total_count = spark.table(TABLE_SOURCE).count()except:
    print(f"\n📈 Total de registros disponíveis: {total_count:,}")

## 3️⃣ Configurar Structured Streaming com maxFilesPerTrigger=1

Agora criamos o streaming que processa **1 arquivo por vez** para demonstração visual.

In [ ]:
# Schema do NYC Taxi (baseado em samples.nyctaxi.trips)
taxi_schema = StructType([
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("pickup_location_id", IntegerType(), True),
    StructField("dropoff_location_id", IntegerType(), True)
])

# Criar streaming DataFrame com maxFilesPerTrigger=1
streaming_df = spark.readStream \
    .schema(taxi_schema) \
    .option("maxFilesPerTrigger", 1) \
    .parquet(INPUT_PATH)

print("✅ Streaming configurado!")
print(f"⚙️  maxFilesPerTrigger: 1 (processa 1 arquivo por vez)")
print(f"📊 É streaming? {streaming_df.isStreaming}")
print("\n📋 Schema:")
streaming_df.printSchema()

## 4️⃣ Enriquecer e Agregar Dados

Adicionamos colunas de análise e criamos agregações por região e tipo de pagamento.

In [ ]:
# Enriquecer dados
enriched_df = streaming_df \
    .withColumn("processing_time", current_timestamp()) \
    .withColumn("file_name", input_file_name()) \
    .withColumn("pickup_date", to_date(col("tpep_pickup_datetime"))) \
    .withColumn("pickup_hour", hour(col("tpep_pickup_datetime"))) \
    .withColumn("trip_duration_minutes", 
                (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60) \
    .withColumn("payment_type_desc", 
                when(col("payment_type") == 1, "Credit Card")
                .when(col("payment_type") == 2, "Cash")
                .when(col("payment_type") == 3, "No Charge")
                .when(col("payment_type") == 4, "Dispute")
                .otherwise("Unknown"))

# Agregações por localização de pickup e tipo de pagamento
aggregated_df = enriched_df \
    .groupBy("pickup_location_id", "payment_type_desc", "pickup_date") \
    .agg(
        count("*").alias("total_trips"),
        sum("trip_distance").alias("total_distance"),
        sum("total_amount").alias("total_revenue"),
        avg("total_amount").alias("avg_fare"),
        avg("trip_distance").alias("avg_distance"),
        avg("trip_duration_minutes").alias("avg_duration_min"),
        min("tpep_pickup_datetime").alias("first_trip"),
        max("tpep_pickup_datetime").alias("last_trip")
    )

print("✅ Pipeline de agregação criado!")
print("\n📊 Métricas calculadas:")
print("  • Total de viagens")
print("  • Distância total percorrida")
print("  • Receita total")
print("  • Ticket médio (fare)")
print("  • Distância média")
print("  • Duração média (minutos)")

## 5️⃣ foreachBatch - Processar e Visualizar Cada Arquivo

Usamos `foreachBatch` para ver o processamento **arquivo por arquivo** em tempo real!

In [ ]:
# Estatísticas acumuladas
batch_stats = {
    'count': 0,
    'total_trips': 0,
    'total_revenue': 0,
    'total_distance': 0
}

def process_batch(batch_df, batch_id):
    """Processa cada batch (arquivo) do streaming"""
    batch_stats['count'] += 1
    
    print("\n" + "=" * 100)
    print(f"📦 BATCH #{batch_stats['count']} | Batch ID: {batch_id}")
    print("=" * 100)
    
    if batch_df.count() == 0:
        print("⚠️  Batch vazio - pulando")
        return
    
    # Estatísticas do batch
    stats = batch_df.agg(
        sum("total_trips").alias("trips"),
        sum("total_revenue").alias("revenue"),
        sum("total_distance").alias("distance")
    ).collect()[0]
    
    batch_stats['total_trips'] += stats.trips if stats.trips else 0
    batch_stats['total_revenue'] += stats.revenue if stats.revenue else 0
    batch_stats['total_distance'] += stats.distance if stats.distance else 0
    
    print(f"\n📊 ESTATÍSTICAS DO BATCH:")
    print(f"   🚕 Viagens: {stats.trips:,}")
    print(f"   💰 Receita: ${stats.revenue:,.2f}")
    print(f"   📏 Distância: {stats.distance:,.2f} milhas")
    
    # Top 5 localizações por receita
    print(f"\n🏆 TOP 5 LOCALIZAÇÕES (Receita):")
    top_locations = batch_df.orderBy(col("total_revenue").desc()).limit(5)
    top_locations.select(
        "pickup_location_id",
        "total_trips",
        "total_revenue",
        "avg_fare"
    ).show(truncate=False)
    
    # Distribuição por tipo de pagamento
    print(f"💳 DISTRIBUIÇÃO POR TIPO DE PAGAMENTO:")
    payment_dist = batch_df.groupBy("payment_type_desc") \
        .agg(
            sum("total_trips").alias("trips"),
            sum("total_revenue").alias("revenue")
        ) \
        .orderBy(col("revenue").desc())
    payment_dist.show(truncate=False)
    
    # Totais acumulados
    print(f"\n📈 TOTAIS ACUMULADOS:")
    print(f"   📦 Batches processados: {batch_stats['count']}")
    print(f"   🚕 Viagens: {batch_stats['total_trips']:,}")
    print(f"   💰 Receita: ${batch_stats['total_revenue']:,.2f}")
    print(f"   📏 Distância: {batch_stats['total_distance']:,.2f} milhas")
    print(f"   🎯 Receita Média/Viagem: ${batch_stats['total_revenue']/max(batch_stats['total_trips'],1):,.2f}")

print("🔧 Função de processamento definida!")

## 6️⃣ EXECUTAR Streaming com foreachBatch

**Execute esta célula e observe o processamento arquivo por arquivo!**

⏱️ Aguarde ~60 segundos para processar todos os 10 arquivos.

In [ ]:
# Resetar estatísticas
batch_stats = {'count': 0, 'total_trips': 0, 'total_revenue': 0, 'total_distance': 0}

# Limpar checkpoint anterior
try:
    dbutils.fs.rm(CHECKPOINT_PATH, True)
except:
    pass

print("🚀 INICIANDO STRUCTURED STREAMING")
print("=" * 100)
print(f"📂 Source: {INPUT_PATH}")
print(f"⚙️  maxFilesPerTrigger: 1")
print(f"⏱️  Trigger: a cada 5 segundos")
print(f"🔖 Checkpoint: {CHECKPOINT_PATH}")
print("=" * 100)

# Iniciar streaming com foreachBatch
query = aggregated_df.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("complete") \
    .option("checkpointLocation", CHECKPOINT_PATH) \
    .trigger(processingTime="5 seconds") \
    .start()

print(f"\n✅ Stream ativo! Query ID: {query.id}")
print("⏳ Processando... Observe os batches abaixo:")

## 7️⃣ Parar o Streaming

**Aguarde ~60 segundos, então execute esta célula para parar e ver o resumo final.**

In [ ]:
import time

print("⏳ Aguardando 60 segundos para processar todos os arquivos...")
time.sleep(60)

query.stop()

print("\n" + "=" * 100)
print("🛑 STREAMING FINALIZADO")
print("=" * 100)
print(f"\n📊 RESUMO FINAL:")
print(f"   📦 Total de batches: {batch_stats['count']}")
print(f"   🚕 Total de viagens: {batch_stats['total_trips']:,}")
print(f"   💰 Receita total: ${batch_stats['total_revenue']:,.2f}")
print(f"   📏 Distância total: {batch_stats['total_distance']:,.2f} milhas")
print(f"   🎯 Receita média/viagem: ${batch_stats['total_revenue']/max(batch_stats['total_trips'],1):,.2f}")
print(f"   📊 Distância média/viagem: {batch_stats['total_distance']/max(batch_stats['total_trips'],1):.2f} milhas")
print("=" * 100)

## 8️⃣ OPÇÃO 2: Gravar em Delta Lake

Alternativa ao foreachBatch: gravar diretamente em Delta Lake para consultas posteriores.

In [ ]:
# Limpar checkpoint anterior
try:
    dbutils.fs.rm(f"{CHECKPOINT_PATH}_delta", True)
    dbutils.fs.rm(OUTPUT_PATH, True)
except:
    pass

print("🚀 Iniciando streaming com saída para Delta Lake...")
print(f"💾 Tabela: {OUTPUT_PATH}")
print(f"🔖 Checkpoint: {CHECKPOINT_PATH}_delta\n")

# Streaming para Delta com agregações
query_delta = aggregated_df.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", f"{CHECKPOINT_PATH}_delta") \
    .trigger(processingTime="5 seconds") \
    .start(OUTPUT_PATH)

print(f"✅ Stream Delta ativo! Query ID: {query_delta.id}")
print(f"💡 Execute a próxima célula após ~60 segundos")

In [ ]:
import time
time.sleep(60)
query_delta.stop()
print("✅ Stream Delta finalizado!")

## 9️⃣ Consultar Tabela Delta

Análise batch dos dados processados pelo streaming.

In [ ]:
# Ler tabela Delta
delta_df = spark.read.format("delta").load(OUTPUT_PATH)

print(f"📊 Tabela Delta carregada!")
print(f"📈 Total de registros: {delta_df.count():,}\n")

print("🏆 TOP 10 - Localizações + Tipo Pagamento (por Receita):")
delta_df.orderBy(col("total_revenue").desc()).limit(10).show(truncate=False)

# Totais gerais
totals = delta_df.agg(
    sum("total_trips").alias("trips"),
    sum("total_revenue").alias("revenue"),
    sum("total_distance").alias("distance")
).collect()[0]

print(f"\n💰 TOTAIS GERAIS:")
print(f"   🚕 Viagens: {totals.trips:,}")
print(f"   💵 Receita: ${totals.revenue:,.2f}")
print(f"   📏 Distância: {totals.distance:,.2f} milhas")
print(f"   🎯 Ticket Médio: ${totals.revenue/totals.trips:,.2f}")

## 🔟 Criar Tabela Gerenciada no Catalog (Opcional)

Cria uma tabela gerenciada para consultas SQL facilitadas.

In [ ]:
# Criar database
spark.sql("CREATE DATABASE IF NOT EXISTS nyctaxi_demo")

# Criar tabela gerenciada
delta_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("nyctaxi_demo.trip_aggregations")

print("✅ Tabela criada: nyctaxi_demo.trip_aggregations")
print("\n📊 Amostra da tabela:")

spark.sql("SELECT * FROM nyctaxi_demo.trip_aggregations LIMIT 10").show(truncate=False)

print("\n💡 Agora você pode consultar com SQL:")
print("   SELECT payment_type_desc, SUM(total_revenue)")
print("   FROM nyctaxi_demo.trip_aggregations")
print("   GROUP BY payment_type_desc")

## 1️⃣1️⃣ Limpeza de Recursos

Remove todos os arquivos temporários criados nesta demonstração.

In [ ]:
print("🧹 Limpando recursos...\n")

# Parar todos os streams ativos
for s in spark.streams.active:
    s.stop()
    print(f"✓ Stream {s.id} parado")

# Remover arquivos
paths = [BASE_PATH]
for path in paths:
    try:
        dbutils.fs.rm(path, True)
        print(f"✓ Removido: {path}")
    except:
        pass

print("\n✅ Limpeza concluída!")

---

## 📚 Conceitos Demonstrados

### Structured Streaming
- **maxFilesPerTrigger=1**: Processa arquivos um por vez (ideal para demonstração visual)
- **foreachBatch**: Controle total sobre cada micro-batch processado
- **Delta Lake Output**: Gravação incremental com ACID transactions
- **Agregações em Streaming**: GROUP BY com múltiplas métricas (count, sum, avg)

### Checkpoint & Fault Tolerance
- Checkpoints garantem processamento exactly-once
- Recuperação automática em caso de falhas
- Estado da agregação mantido entre micro-batches

### Performance
- Particionamento dos dados de entrada para paralelização
- Agregações otimizadas com Catalyst optimizer
- Delta Lake para leitura/escrita eficiente

---

## 🎓 Para Seus Alunos

**Conceitos-chave demonstrados:**

1. **Processamento Incremental**: Cada arquivo é um micro-batch
2. **Agregações Stateful**: Mantém estado acumulado (complete mode)
3. **Observabilidade**: Métricas detalhadas batch por batch
4. **Delta Lake**: Camada de armazenamento confiável para streaming
5. **Dados Reais**: NYC Taxi demonstra aplicações práticas

**Próximos passos:**
1. Modificar agregações (diferentes GROUP BY)
2. Adicionar filtros (apenas viagens > 5 milhas)
3. Window functions (agregações por janela de tempo)
4. Joins com dados estáticos (ex: informações de zonas)
5. Integrar com fontes reais (quando sair do Free Edition)

---

## 📖 Recursos Adicionais

**Documentação Oficial:**
- [Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Structured Streaming](https://docs.databricks.com/structured-streaming/index.html)
- [Delta Lake Streaming](https://docs.delta.io/latest/delta-streaming.html)
- [Unity Catalog Datasets](https://docs.databricks.com/discover/databricks-datasets.html)

**Databricks Free Edition:**
- Use `display()` para visualizações interativas
- Workspace → Data → Catalog Explorer para navegar tabelas
- Monitor: Workspace → Compute → seu cluster → Streaming Queries

**Integração com Git:**
1. Workspace → Git Folders → Add Repo
2. Clone: `https://github.com/seu-usuario/seu-repo`
3. Commits/Push direto da UI do Databricks
4. Branches: Use Git Folders para alternar branches

---

## ✅ Checklist de Execução

- [ ] Célula 2: Ler dados de `samples.nyctaxi.trips`
- [ ] Célula 3: Particionar em 10 arquivos Parquet
- [ ] Célula 4: Configurar streaming com maxFilesPerTrigger=1
- [ ] Célula 5: Definir agregações
- [ ] Célula 6: Definir função foreachBatch
- [ ] Célula 7: **EXECUTAR streaming** (observe os batches!)
- [ ] Célula 8: Parar e ver resumo final
- [ ] Célula 9-11: (Opcional) Gravar em Delta Lake
- [ ] Célula 12: (Opcional) Consultar resultados
- [ ] Célula 13: (Opcional) Criar tabela gerenciada
- [ ] Célula 14: Limpar recursos

**🎯 Perfeito para ensinar os fundamentos de Structured Streaming com dados REAIS!**